In [2]:
from histdata import download_hist_data as dl
from histdata.api import Platform as P, TimeFrame as TF
import pandas as pd
import numpy as np
from datetime import datetime as dt
import zipfile
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import gc
import sys, os, shutil

# Functions

In [3]:
# Disable
def blockPrint():
    sys.stdout = open(os.devnull, 'w')

# Restore
def enablePrint():
    sys.stdout = sys.__stdout__

In [4]:
def dl_data(year, pair, platform=P.GENERIC_ASCII, time_frame=TF.ONE_MINUTE):
    year = str(year)
    zip_name = dl(year=year, month=None, pair=pair, platform=platform, time_frame=time_frame)[2:]
    with zipfile.ZipFile(zip_name,"r") as zip_ref:
        zip_ref.extractall("temp")
    csv_name = zip_name.replace('zip','csv')
    df = pd.read_csv(f'temp/{csv_name}',header=None,sep=';')
    return df

In [5]:
def connect_db(db_url):
    engine = create_engine(db_url, echo=False)
    if not database_exists(engine.url):
        create_database(engine.url)
    return engine

In [6]:
def clean_data(data):
    columns = ['Datetime','Open','High','Low','Close','Volume']
    data.columns = columns
    data['Datetime'] = pd.to_datetime(data['Datetime'],format='%Y%m%d %H%M%S')
    data.sort_index(inplace=True)
    return data

In [7]:
def clear_temp(folder):
    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))

# Main

In [8]:
pairs = ['audusd','eurusd','gbpusd','nzdusd','usdcad','usdchf','usdjpy']

db_url = 'sqlite:///fx_min_data.db'
db_engine = connect_db(db_url)

for pair in pairs:
    # download data by year
    data_list = list()
    for year in range(2000,2024):
        try:
            data_list.append(dl_data(year,pair))
        except:
            print(f'{pair} data in {year} not found.')

    # clean data and save to sql
    data = clean_data(pd.concat(data_list,axis=0))
    data.to_sql(pair, con=db_engine)
    
    # free memory
    del data_list
    del data
    gc.collect()

    # delete temp files
    clear_temp('temp')

https://www.histdata.com/download-free-forex-historical-data/?/ascii/1-minute-bar-quotes/audusd/2000
{'tk': '55a31ada7f44194623c4f93940a19622', 'date': '2000', 'datemonth': '2000', 'platform': 'ASCII', 'timeframe': 'M1', 'fxpair': 'AUDUSD'}
Wrote to .\DAT_ASCII_AUDUSD_M1_2000.zip
https://www.histdata.com/download-free-forex-historical-data/?/ascii/1-minute-bar-quotes/audusd/2001
{'tk': '5ed57d02a7f1aee7dde2d506ef03b1e3', 'date': '2001', 'datemonth': '2001', 'platform': 'ASCII', 'timeframe': 'M1', 'fxpair': 'AUDUSD'}
Wrote to .\DAT_ASCII_AUDUSD_M1_2001.zip
https://www.histdata.com/download-free-forex-historical-data/?/ascii/1-minute-bar-quotes/audusd/2002
{'tk': '6eb7809341fdf77e200f963564488676', 'date': '2002', 'datemonth': '2002', 'platform': 'ASCII', 'timeframe': 'M1', 'fxpair': 'AUDUSD'}
Wrote to .\DAT_ASCII_AUDUSD_M1_2002.zip
https://www.histdata.com/download-free-forex-historical-data/?/ascii/1-minute-bar-quotes/audusd/2003
{'tk': '4ac032c232bd91ffe33130cc7fff5ec5', 'date': '2003

In [12]:
pair = 'usdjpy'
sample_sql_database = db_engine.execute(f"SELECT * FROM {pair} LIMIT 10").fetchall()

print(sample_sql_database)

[(0, '2000-05-30 17:58:00.000000', 106.6, 106.6, 106.6, 106.6, 0), (0, '2022-01-02 17:00:00.000000', 115.039, 115.04, 115.039, 115.04, 0), (0, '2021-01-03 17:00:00.000000', 103.097, 103.16, 103.097, 103.16, 0), (0, '2002-01-02 01:26:00.000000', 131.52, 131.77, 131.52, 131.77, 0), (0, '2020-01-01 17:00:00.000000', 108.73, 108.73, 108.728, 108.728, 0), (0, '2019-01-01 17:00:00.000000', 109.648, 109.65, 109.648, 109.65, 0), (0, '2018-01-01 17:00:00.000000', 112.645, 112.645, 112.645, 112.645, 0), (0, '2003-01-01 18:59:00.000000', 118.8, 118.8, 118.79, 118.79, 0), (0, '2017-01-02 02:00:00.000000', 116.858, 116.87, 116.858, 116.87, 0), (0, '2016-01-03 17:00:00.000000', 120.172, 120.205, 120.171, 120.205, 0)]
